기본 설정?

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip /content/drive/MyDrive/DACON/Computer\ Vision\ 이상치\ 탐지\ 알고리즘\ 경진대회/open.zip # 자신의 train.zip 


Archive:  /content/drive/MyDrive/DACON/Computer Vision 이상치 탐지 알고리즘 경진대회/open.zip
  inflating: open/sample_submission.csv  
 extracting: open/test.zip           
  inflating: open/test_df.csv        
 extracting: open/train.zip          
  inflating: open/train_df.csv       


In [31]:
import os
print(os.getcwd())

/content


zip파일 풀기

In [32]:
!unzip /content/open/test.zip
!unzip /content/open/train.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: test/21432.png          
  inflating: test/21433.png          
  inflating: test/21434.png          
  inflating: test/21435.png          
  inflating: test/21436.png          
  inflating: test/21437.png          
  inflating: test/21438.png          
  inflating: test/21439.png          
  inflating: test/21440.png          
  inflating: test/21441.png          
  inflating: test/21442.png          
  inflating: test/21443.png          
  inflating: test/21444.png          
  inflating: test/21445.png          
  inflating: test/21446.png          
  inflating: test/21447.png          
  inflating: test/21448.png          
  inflating: test/21449.png          
  inflating: test/21450.png          
  inflating: test/21451.png          
  inflating: test/21452.png          
  inflating: test/21453.png          
  inflating: test/21454.png          
  inflating: test/21455.png          
  inflating: test/21456.png          
  inflating: t

In [3]:
!pip install timm

     |████████████████████████████████| 431 kB 12.5 MB/s 


In [41]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time


device = torch.device('cuda')

이미지 이름 리스트에 저장 <class:list>

In [42]:
train_png = sorted(glob('train/*.png'))
test_png = sorted(glob('test/*.png'))
print(train_png)

['train/10000.png', 'train/10001.png', 'train/10002.png', 'train/10003.png', 'train/10004.png', 'train/10005.png', 'train/10006.png', 'train/10007.png', 'train/10008.png', 'train/10009.png', 'train/10010.png', 'train/10011.png', 'train/10012.png', 'train/10013.png', 'train/10014.png', 'train/10015.png', 'train/10016.png', 'train/10017.png', 'train/10018.png', 'train/10019.png', 'train/10020.png', 'train/10021.png', 'train/10022.png', 'train/10023.png', 'train/10024.png', 'train/10025.png', 'train/10026.png', 'train/10027.png', 'train/10028.png', 'train/10029.png', 'train/10030.png', 'train/10031.png', 'train/10032.png', 'train/10033.png', 'train/10034.png', 'train/10035.png', 'train/10036.png', 'train/10037.png', 'train/10038.png', 'train/10039.png', 'train/10040.png', 'train/10041.png', 'train/10042.png', 'train/10043.png', 'train/10044.png', 'train/10045.png', 'train/10046.png', 'train/10047.png', 'train/10048.png', 'train/10049.png', 'train/10050.png', 'train/10051.png', 'train/1005

In [43]:
train_y = pd.read_csv("open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [44]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (416, 416))
    return img

In [36]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 2154/2154 [01:11<00:00, 29.94it/s]


In [37]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        img = transforms.ToTensor()(img)
        if self.mode=='test':
            pass
        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [38]:
batch_size = 32
epochs = 25

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [39]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 



best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth


epoch : 1/25    time : 166s/3975s
TRAIN    loss : 0.97557    f1 : 0.23257
epoch : 2/25    time : 166s/3812s
TRAIN    loss : 0.53964    f1 : 0.38477


KeyboardInterrupt: ignored

In [ ]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [ ]:
submission = pd.read_csv("open/sample_submission.csv")

submission["label"] = f_result

submission

In [ ]:
submission.to_csv("baseline.csv", index = False)